## Load modules

In [1]:
push!(LOAD_PATH, "./dvi/src")

addprocs(int(CPU_CORES / 2))

using DiscreteValueIteration
using SCAs, SCAIterators, SCAConst
using GridInterpolations

mdp = SCA()

SCA(5446,36,StateIterator([-2000.0,-1600.0,-1200.0,-800.0,-400.0,0.0,400.0,800.0,1200.0,1600.0,2000.0],[-2000.0,-1600.0,-1200.0,-800.0,-400.0,0.0,400.0,800.0,1200.0,1600.0,2000.0],[0.0,1.5708,3.14159,4.71239,6.28319],[10.0,15.0,20.0],[10.0,15.0,20.0],1,1,1,1,1,5446),ActionIterator([:right20,:right10,:straight,:left10,:left20,:clearOfConflict],1,1,36),RectangleGrid with 5445 points)

## Check size of MDP

In [2]:
function getBytes(x)
   total = 0;
   fieldNames = typeof(x).names;
   if fieldNames == ()
      return sizeof(x);
   else
     for fieldName in fieldNames
        total += getBytes(getfield(x,fieldName));
     end
     return total;
   end
end

println("mdp of type ", typeof(mdp), " takes up ", getBytes(mdp) / 1000.0, " kB")

mdp of type SCA takes up 1.768 kB


## Informal validation of state and action iterators and transition function

In [3]:
testActions = ActionIterator(Actions)
actionCount = 0
for action in testActions
    actionCount += 1
    if actionCount == 1 || actionCount == NActions
        println("action #", actionCount, ": ", typeof(action), " = ", action)
    end # if
end # for action
println("nActions = ", NActions)
println("actionCount = ", actionCount, "\n")

testStates = StateIterator(Xs, Ys, Bearings, Speeds, Speeds)
stateCount = 0
for state in testStates
    stateCount += 1
    if stateCount == 1 || stateCount == NStates
        println("state #", stateCount, ": ", typeof(state), " = ", state)
    end # if
end # for state
println("nStates = ", NStates)
println("stateCount = ", stateCount)

action #1: Action = Action(:right20,:right20)
action #36: Action = Action(:clearOfConflict,:clearOfConflict)
nActions = 36
actionCount = 36

state #1: State = State(-2000.0,-2000.0,0.0,10.0,10.0,false)
state #5446: State = State(-2000.0,-2000.0,0.0,10.0,10.0,true)
nStates = 5446
stateCount = 5446


In [4]:
nextStateIndices, probs = nextStates(mdp, 1, 15)
println("next state indices:\n", nextStateIndices, "\n")
println("probabilities:\n", probs, "\n")
println("probabilities sum to ", sum(probs))

next state indices:
[1,1,2,5446,5446,1,2,1816,1817,5446,5446,5446,5446]

probabilities:
[0.3333333333333333,0.08125,0.0020833333333333355,0.08333333333333334,0.08333333333333334,0.04875,0.0012500000000000013,0.03250000000000001,0.0008333333333333341,0.08333333333333334,0.08333333333333334,0.08333333333333334,0.08333333333333334]
probabilities sum to 1.0


## Parallel solution

In [5]:
numProcs = int(CPU_CORES / 2)
solver = ParallelSolver(
    numProcs,
    maxIterations = 1000,
    tolerance = 1e-6,
    gaussSiedel = false,
    includeV = true,
    includeQ = true,
    includeA = true)

ParallelSolver(2,None[],1000,1.0e-6,false,true,true,true)

In [6]:
policy = solve(solver, mdp, verbose = true)
println("")

Iter 1: resid = 1.020e+06, runtime = 1.258e+01, net runtime = 1.258e+01
Iter 2: resid = 9.199e+05, runtime = 1.146e+01, net runtime = 2.404e+01
Iter 3: resid = 6.774e+05, runtime = 1.152e+01, net runtime = 3.556e+01
Iter 4: resid = 4.371e+05, runtime = 1.143e+01, net runtime = 4.699e+01
Iter 5: resid = 2.453e+05, runtime = 1.148e+01, net runtime = 5.847e+01
Iter 6: resid = 1.371e+05, runtime = 1.188e+01, net runtime = 7.034e+01
Iter 7: resid = 7.716e+04, runtime = 1.151e+01, net runtime = 8.185e+01
Iter 8: resid = 4.358e+04, runtime = 1.140e+01, net runtime = 9.325e+01
Iter 9: resid = 2.436e+04, runtime = 1.148e+01, net runtime = 1.047e+02
Iter 10: resid = 1.350e+04, runtime = 1.149e+01, net runtime = 1.162e+02
Iter 11: resid = 7.425e+03, runtime = 1.150e+01, net runtime = 1.277e+02
Iter 12: resid = 4.063e+03, runtime = 1.146e+01, net runtime = 1.392e+02
Iter 13: resid = 2.214e+03, runtime = 1.154e+01, net runtime = 1.507e+02
Iter 14: resid = 1.202e+03, runtime = 1.181e+01, net runtime

## Check against reference solution by visual inspection

In [ ]:
push!(LOAD_PATH, "../src/")
using Pairwise, Multiagent, HDF5, JLD

d = get_pomdp()
refQ = load("../data/alpha.jld", "alpha")
solQ = policy.Q'
save("../data/par-alpha.jld", "solQ", solQ)

### Reference policy plot

In [ ]:
viz_pairwise_policy(d, refQ)

### Parallel solver policy plot

In [ ]:
function sharray2array(sharray::SharedArray{Float64, 2})
    result = zeros(solQ.dims)
    for i = 1:solQ.dims[1]
        for j = 1:solQ.dims[2]
            result[i, j] = sharray[i, j]
        end # for j
    end # for i
    return result
end # function sharray2array

In [ ]:
viz_pairwise_policy(d, sharray2array(solQ))